In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ops import pred_reconctruct
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from model.callbacks import UpdateWeights
from model.losses import WBCE
import matplotlib.pyplot as plt
import time
import tensorflow as tf
import os
import gc
import json
import importlib
import numpy as np

## Parameters

In [3]:
with open(f'00_params.json') as param_file:
    params = json.load(param_file)

img_path = 'imgs' 
n_opt_layer = 26 #number of OPT layers, used to split de input data between OPT and SAR

number_class = 3
weights = params['weights']
times= params['times']
exp = params['exp']

overlap = params['overlap']
patch_size = params['patch_size']
batch_size = params['batch_size']

img_type = params['img_type']
cond = params['cond']
test_cond = params['test_cond']

nb_filters = params['nb_filters']

method = params['method']
module = importlib.import_module('model.models')
exp_model = getattr(module, method)

grid_size = params['grid_size']

tiles_tr = params['tiles_tr']
tiles_val = params['tiles_val']

In [4]:
print(f'Experiment: {exp}')
print(f'Model: {method}')
print(f'Test Cond: {test_cond}')

Experiment: 8
Model: CrossFusion_ResUNET
Test Cond: no_cloudy


## Load test file

In [5]:
image_array = np.load(os.path.join(img_path, f'fus_stack_{test_cond}.npy'))
if img_type == 'OPT':
    image_array = image_array[:, :, :n_opt_layer]
    
if img_type == 'SAR':
    image_array = image_array[:, :, n_opt_layer:]
    
    
print('Image stack:', image_array.shape)
h_, w_, channels = image_array.shape

Image stack: (17730, 9203, 30)


## Create tiles mask

In [6]:
path_exp = os.path.join(img_path, 'experiments', f'exp_{exp}')
path_models = os.path.join(path_exp, 'models')
path_maps = os.path.join(path_exp, 'pred_maps')

if not os.path.exists(path_exp):
    os.makedirs(path_exp)   
if not os.path.exists(path_models):
    os.makedirs(path_models)   
if not os.path.exists(path_maps):
    os.makedirs(path_maps)

## Generating predictions maps

In [8]:
input_shape = (patch_size, patch_size, channels)
time_ts = []
n_pool = 3
n_rows = 12#6
n_cols = 6#3
rows, cols = image_array.shape[:2]
pad_rows = rows - np.ceil(rows/(n_rows*2**n_pool))*n_rows*2**n_pool
pad_cols = cols - np.ceil(cols/(n_cols*2**n_pool))*n_cols*2**n_pool
print(pad_rows, pad_cols)

npad = ((0, int(abs(pad_rows))), (0, int(abs(pad_cols))), (0, 0))
image1_pad = np.pad(image_array, pad_width=npad, mode='reflect')

h, w, c = image1_pad.shape
patch_size_rows = h//n_rows
patch_size_cols = w//n_cols
num_patches_x = int(h/patch_size_rows)
num_patches_y = int(w/patch_size_cols)

input_shape=(patch_size_rows,patch_size_cols, c)

new_model = exp_model(nb_filters, number_class, n_opt_layer)
new_model.build((None,)+input_shape)


adam = Adam(lr = 1e-3 , beta_1=0.9)
loss = WBCE(weights = weights)
optimizers = [
        Adam(lr = 1e-4 , beta_1=0.9),
        Adam(lr = 1e-4 , beta_1=0.9),
        Adam(lr = 1e-4 , beta_1=0.9)
    ]
new_model.compile(optimizers=optimizers, loss=loss, metrics=['accuracy'])

for tm in range(0,times):
    
    new_model.load_weights(os.path.join(path_models, f'{method}_{tm}.h5'))
    
    start_test = time.time()
    patch_opt = []
    patch_sar = []
    patch_fus = []
    
    for i in range(0,num_patches_y):
        for j in range(0,num_patches_x):
            patch = image1_pad[patch_size_rows*j:patch_size_rows*(j+1), patch_size_cols*i:patch_size_cols*(i+1), :]
            pred_opt, pred_sar, pred_fus = new_model.predict(np.expand_dims(patch, axis=0))
            del patch 
            patch_opt.append(pred_opt[:,:,:,1])
            patch_sar.append(pred_sar[:,:,:,1])
            patch_fus.append(pred_fus[:,:,:,1])
            del pred_opt, pred_sar, pred_fus
    end_test =  time.time() - start_test

    patches_pred_opt = np.asarray(patch_opt).astype(np.float32)
    patches_pred_sar = np.asarray(patch_sar).astype(np.float32)
    patches_pred_fus = np.asarray(patch_fus).astype(np.float32)

    prob_recontructed_opt = pred_reconctruct(h, w, num_patches_x, num_patches_y, patch_size_rows, patch_size_cols, patches_pred_opt)
    prob_recontructed_sar = pred_reconctruct(h, w, num_patches_x, num_patches_y, patch_size_rows, patch_size_cols, patches_pred_sar)
    prob_recontructed_fus = pred_reconctruct(h, w, num_patches_x, num_patches_y, patch_size_rows, patch_size_cols, patches_pred_fus)

    del patches_pred_opt, patches_pred_sar, patches_pred_fus
    
    np.save(os.path.join(path_maps, f'prob_opt_{tm}.npy'),prob_recontructed_opt) 
    np.save(os.path.join(path_maps, f'prob_sar_{tm}.npy'),prob_recontructed_sar) 
    np.save(os.path.join(path_maps, f'prob_fus_{tm}.npy'),prob_recontructed_fus) 

    time_ts.append(end_test)
    print(f'model {tm}: {end_test:.2f}')
    del prob_recontructed_opt, prob_recontructed_sar, prob_recontructed_fus
    gc.collect()
    #del model
time_ts_array = np.asarray(time_ts)
# Save test time
np.save(os.path.join(path_exp, 'metrics_ts.npy'), time_ts_array)
del image_array
gc.collect()

-30.0 -13.0
model 0: 30.95
model 1: 22.91
model 2: 22.79
model 3: 22.36
model 4: 22.50


0

## Compute the mean of the predictions

In [9]:
prob_rec_opt = np.zeros((image1_pad.shape[0],image1_pad.shape[1], times))
prob_rec_sar = np.zeros((image1_pad.shape[0],image1_pad.shape[1], times))
prob_rec_fus = np.zeros((image1_pad.shape[0],image1_pad.shape[1], times))

for tm in range (0, times):
    print(tm)
    prob_rec_opt[:,:,tm] = np.load(os.path.join(path_maps, f'prob_opt_{tm}.npy')).astype(np.float32)
    prob_rec_sar[:,:,tm] = np.load(os.path.join(path_maps, f'prob_sar_{tm}.npy')).astype(np.float32)
    prob_rec_fus[:,:,tm] = np.load(os.path.join(path_maps, f'prob_fus_{tm}.npy')).astype(np.float32)

mean_prob_opt = np.mean(prob_rec_opt, axis = -1)
mean_prob_sar = np.mean(prob_rec_sar, axis = -1)
mean_prob_fus = np.mean(prob_rec_fus, axis = -1)

np.save(os.path.join(path_maps, f'prob_mean_opt.npy'), mean_prob_opt)
np.save(os.path.join(path_maps, f'prob_mean_sar.npy'), mean_prob_sar)
np.save(os.path.join(path_maps, f'prob_mean_fus.npy'), mean_prob_fus)


for tm in range (0, times):
    os.remove(os.path.join(path_maps, f'prob_opt_{tm}.npy'))
    os.remove(os.path.join(path_maps, f'prob_sar_{tm}.npy'))
    os.remove(os.path.join(path_maps, f'prob_fus_{tm}.npy'))


0
1
2
3
4
